In [1]:
import numpy
import seaborn as sns
import pandas as pd
import matplotlib
from matplotlib import *
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [34]:
in_labels = pd.read_csv('data/input-data-labels.csv',encoding='utf-8')
in_labelling = pd.read_csv('data/input-data-labeling.csv',encoding='utf-8')

In [3]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [4]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000) 

In [5]:
name_features = vectorizer.fit_transform(in_labelling['name'])
name_features = name_features.toarray()

In [6]:
numpy.count_nonzero(name_features)

606124

In [7]:
name_df = pd.DataFrame(name_features)

In [8]:
name_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
del in_labelling['name']
del in_labelling['description']
del in_labelling['id']

In [10]:
label_targets = in_labelling['labels']

In [35]:
label_targets2 = in_labelling['labels']

In [11]:
del in_labelling['labels']

In [12]:
in_labelling.head()

,price,shop,brand
0,84.949997,Herrenausstatter,Herrenausstatter.de
1,24.950001,Herrenausstatter,Jockey
2,24.950001,Herrenausstatter,Jockey
3,39.950001,Herrenausstatter,Jockey
4,19.950001,Herrenausstatter,HOM


In [13]:
from sklearn import preprocessing
# Discreet value integer encoder
label_encoder = preprocessing.LabelEncoder()

In [14]:
in_labelling['shop'] = label_encoder.fit_transform(in_labelling['shop'])
in_labelling['brand'] = label_encoder.fit_transform(in_labelling['brand'])

In [15]:
in_labelling.head()

,price,shop,brand
0,84.949997,117,1713
1,24.950001,117,1941
2,24.950001,117,1941
3,39.950001,117,1941
4,19.950001,117,1645


In [16]:
feature_df = pd.concat([in_labelling,name_df],axis=1)

In [17]:
def get_level1(df_text):
    split_text = df_text.split(',')
    return split_text[0]

In [18]:
label_targets = label_targets.apply(get_level1)

In [19]:
label_targets = label_encoder.fit_transform(label_targets)

In [20]:
label_targets[:200]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 3, 3, 1, 3, 1, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 3, 1, 1, 3, 1, 3, 1, 1, 1, 1, 3, 0, 3, 1, 1, 1, 1, 1,
       1, 3, 3, 3, 3, 1, 1, 3, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 1, 0, 0, 3, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,
       1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 3, 1, 3, 3, 1, 3, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1,
       1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [21]:
from sklearn import cross_validation
from sklearn import ensemble

In [22]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(feature_df,label_targets,test_size=0.2)

In [23]:
pa_classifier = ensemble.RandomForestClassifier()

In [24]:
pa_classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0)

In [25]:
test_pred = pa_classifier.predict(X_test)

In [26]:
from sklearn import metrics
metrics.accuracy_score(y_test,test_pred)

0.97547499999999998

In [27]:
l1_feature = pa_classifier.predict(feature_df)

In [28]:
metrics.accuracy_score(label_targets,l1_feature)

0.99393500000000001

In [29]:
l1_df = pd.Series(l1_feature)
l1_df.name='level1'

In [30]:
l2_feature_df = pd.concat([feature_df,l1_df],axis=1)

In [31]:
l2_feature_df.head()

,price,shop,brand,0,1,2,3,4,5,6,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,level1
0,84.949997,117,1713,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,24.950001,117,1941,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,24.950001,117,1941,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,39.950001,117,1941,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,19.950001,117,1645,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
def get_level2(df_text):
    split_text = df_text.split(',')
    return split_text[1]

In [36]:
l2_label_targets = label_targets2.apply(get_level2)

In [ ]:
l2_label_targets = label_encoder.fit_transform(label_targets)

In [ ]:
l2_X_train,l2_X_test,l2_y_train,l2_y_test = train_test_split(l2_feature_df,l2_label_targets,test_size=0.2)